In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook



import os as os

import matplotlib.pyplot as plt

def imshow(img, cmap=None):
    plt.close('all')
    plt.figure()
    plt.imshow(img, cmap=cmap)
    plt.show()
    
import scipy.ndimage as ndimage
from scipy import misc
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import cv2
import scipy.signal as sig
from scipy.integrate import simpson
from IPython.display import SVG, display, IFrame, HTML
%matplotlib notebook

In [ ]:
def createPillobxResponse(r, x_size, y_size):
    X,Y = np.meshgrid(np.arange(-1*np.floor(x_size/2),np.floor(x_size/2)+1,1), np.arange(-1*np.floor(y_size/2),np.floor(y_size/2)+1,1))
    psf = np.zeros_like(X)
    psf[np.sqrt(X**2+Y**2) <= r] = 1
    return psf, X, Y
def fourier_conv(img:np.ndarray, kernel:np.ndarray) -> np.ndarray:
    
    img_fft = np.fft.fft2(img)
    
    kernel_padded = pad_like(kernel, img)
    kernel_padded_fft = np.fft.fft2(np.fft.ifftshift(kernel_padded))
    
    result = np.real(np.fft.ifft2(img_fft * kernel_padded_fft))
    
        
    return result
def mag_square(inp : np.ndarray):
    return inp.real**2 + inp.imag**2
def pad_like(inp : np.ndarray, like : np.ndarray):
    to_pad = tuple(np.int32((np.array(like.shape) - np.array(inp.shape))/2))
    return np.pad(inp, ((to_pad[0], to_pad[0]), (to_pad[1], to_pad[1])))
def make_odd_shapes(inp : np.ndarray) -> np.ndarray:
    if (inp.shape[0] % 2 == 0):
        inp = inp[0:-1,:]
    if (inp.shape[1] % 2 == 0):
        inp = inp[:,0:-1]
    return inp

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inverse-problems-part-2" data-toc-modified-id="Inverse-problems-part-2-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Inverse problems part 2</a></span><ul class="toc-item"><li><span><a href="#Half-quadratic-splitting-method" data-toc-modified-id="Half-quadratic-splitting-method-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Half-quadratic splitting method</a></span></li><li><span><a href="#Alternating-direction-method-of-multipliers" data-toc-modified-id="Alternating-direction-method-of-multipliers-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Alternating direction method of multipliers</a></span></li></ul></li></ul></div>

# Inverse problems part 2

In this exercise we will try to implement some of the more advanced deconvolution algorithms on our own.

Test your filter implementations for different convolution kernels and apply different amounts of random noise to the convolution result before applying the deconvolution algorithms.

##### Preparation of data for experiments

In [ ]:
s = make_odd_shapes(np.float32(misc.face(gray=True)))
h,_,_ = createPillobxResponse(20, 51,51)
h = 1/np.sum(h) * h
g = ndimage.convolve(s,h, mode='wrap')
noises = []   # List for observations
sigmas = np.float32(np.array([0,2,4,8,16,32]))  # Standard deviations for Gaussian noise
for sigma in sigmas:
    cur_img = g + np.random.normal(0, sigma, g.shape)
    noises.append(cur_img)

## Half-quadratic splitting method

Implement the HQS method for deconvolution and try different regularizers:
* Promote smoothness (i.e., minimize $\left\| \mathbf{x}  \right\|_2$),
* promote sparseness (i.e., minimize $\left\| \mathbf{x}  \right\|_1$),
* anisotropic total variation,
* isotropic total variation.

## Alternating direction method of multipliers

Implement the ADMM method for deconvolution and also try the above regularizers.